# DenseNet Model Implementation

In [2]:
import os
import glob
import cv2
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
from torch.utils.data import random_split
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
import sklearn
from PIL import Image
import zipfile
import urllib.request
import os.path
from IPython.display import display

# Set GPU access

In [3]:
if torch.backends.mps.is_available(): # Check if PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
    print(f"MPS is available!")
    if torch.backends.mps.is_built():
        print(f"MPS (Metal Performance Shader) is built in!")    
    device = "mps"
elif torch.cuda.is_available():
    print(f"CUDA is available!")
    device = "cuda"
else:
    print(f"Only CPU is available!")
    device = "cpu"
print(f"Using device: {device}")

CUDA is available!
Using device: cuda


Parameters Definition

In [4]:
#Parameters definitions
smartDownsampling = False
nImLevelsData = 9
useImLevels = 8 # we do not use background 
nClasses = 2
downImageSize = 224
imagesFromEachClass = 40
trainTestSplit = 0.8 #Typical 0.8 0.2, but for better evaluation we may need different split
trainShuffleFlag = False
nEpochs = 15 #120 Epochs needed for full convergence

# Helper functions

In [5]:
def create_image_stack(images, labels):
    
    image_count = len(images) // nImLevelsData  

    stacked_data = []
    stacked_labels = []

    normalize = transforms.Normalize([0.485], [0.229]) 

    for i in range(image_count):
        start_index = i * nImLevelsData
        end_index = start_index + nImLevelsData

        single_stack = images[start_index+1:end_index] # only 8 images without background image

        image_stack = np.stack([normalize(torch.from_numpy(img.astype(np.float32))) for img in single_stack])

        image_stack = torch.from_numpy(image_stack)
        
        stacked_data.append(image_stack)
        
        stacked_labels.append(labels[start_index])
    
    return stacked_data, stacked_labels

In [21]:
def compute_score_with_logits(logits, labels):
    logitsM = torch.max(logits, 1)[1].data # argmax
    if device == "mps":
        one_hots = torch.zeros(*labels.size()).to(device)
    elif device == "cuda":
        one_hots = torch.zeros(*labels.size()).cuda()
    else:
        one_hots = torch.zeros(*labels.size())
    one_hots.scatter_(1, logitsM.view(-1, 1), 1)
    scores = (one_hots * labels)

    return scores

In [7]:
def tile(a, dim, n_tile):
    init_dim = a.size(dim)
    repeat_idx = [1] * a.dim()
    repeat_idx[dim] = n_tile
    a = a.repeat(*(repeat_idx))
    order_index = torch.LongTensor(np.concatenate([init_dim * np.arange(n_tile) + i for i in range(init_dim)]))
    return torch.index_select(a, dim, order_index)

# Preprocessing

In [8]:
class DataPreprocessing(Dataset):
    
    # classPaths --> className: [path, #training_samples]
    classPaths = {
    
    'AGE_RMD': ['../../Images/CT_RETINA/AGE_RMD_55/AR1-45_9Levels/', imagesFromEachClass],
    'CSR':     ['../../Images/CT_RETINA/CSR_102/CR1-80_9Levels/', imagesFromEachClass],
    'DIABETR': [ '../../Images/CT_RETINA/DIABETR_107/DR1-83_9Levels/', imagesFromEachClass],
    'MACHOLE': ['../../Images/CT_RETINA/MACHOLE_102/MH1-80_9Levels/', imagesFromEachClass],
    'NORMAL':  ['../../Images/CT_RETINA/NORMAL_206/NR1-160_9Levels/', imagesFromEachClass] 
    
        }

    # classEncoding --> className: label_tensor defition
    if nClasses == 2:
        classEncoding = {
            'AGE_RMD': torch.FloatTensor([1, 0]),
            'NORMAL':  torch.FloatTensor([0, 1])
            }                
    elif nClasses == 3:
        classEncoding = {
            'AGE_RMD': torch.FloatTensor([1, 0, 0]),
            'DIABETR': torch.FloatTensor([0, 1, 0]),
            'NORMAL':  torch.FloatTensor([0, 0, 1])
            }
    elif nClasses == 4:
        classEncoding = {
            'AGE_RMD': torch.FloatTensor([1, 0, 0, 0]),
            'DIABETR': torch.FloatTensor([0, 1, 0, 0]),
            'MACHOLE': torch.FloatTensor([0, 0, 1, 0]),
            'NORMAL':  torch.FloatTensor([0, 0, 0, 1])
            }
    elif nClasses == 5:
        classEncoding = {
            'AGE_RMD': torch.FloatTensor([1, 0, 0, 0, 0]),
            'CSR':     torch.FloatTensor([0, 1, 0, 0, 0]),
            'DIABETR': torch.FloatTensor([0, 0, 1, 0, 0]),
            'MACHOLE': torch.FloatTensor([0, 0, 0, 1, 0]),
            'NORMAL':  torch.FloatTensor([0, 0, 0, 0, 1])
            }
    
    def __init__(self,classEncoding=classEncoding, classPaths=classPaths):
        
        self.image_paths = []
        self.labels = []
        self.images = []
    
        # image paths
        for imCoreName in (classEncoding.keys()):
            temp_paths = []
            for directoryPath in glob.glob(classPaths[imCoreName][0]):
                for imgPath in glob.glob(os.path.join(directoryPath, "*.jpg")):
                    temp_paths.append(imgPath)


            # labels 
            labels_list = [imCoreName] * classPaths[imCoreName][1] * nImLevelsData

            for label in labels_list:
                if nClasses == 2:
                    labelTensor = torch.FloatTensor([0, 0])
                elif nClasses == 3:
                    labelTensor = torch.FloatTensor([0, 0, 0])
                elif nClasses == 4:
                    labelTensor = torch.FloatTensor([0, 0, 0, 0])
                elif nClasses == 5:
                    labelTensor = torch.FloatTensor([0, 0, 0, 0, 0])

                labelTensor = labelTensor.add(classEncoding[label])
                self.labels.append(labelTensor)

            img_paths = temp_paths[:classPaths[imCoreName][1] * 9] 
            self.image_paths.append(img_paths)

            for image_path in img_paths:
                img = cv2.imread(image_path,0) 
                img = cv2.resize(img, (downImageSize, downImageSize), interpolation = cv2.INTER_AREA)
                img = np.reshape(img, (*img.shape, 1))
                img = np.transpose(img, (2, 0, 1))
                self.images.append(img)
              
              
        self.image_paths = [y for x in self.image_paths for y in x]
        self.stacked_images, self.stacked_labels = create_image_stack(self.images, self.labels)  

                
    def __getitem__(self, index):
        # preprocess and return single image stack of dim 8*1*224*224
        
        return self.stacked_images[index], self.stacked_labels[index]
    
    def __len__(self):

        return len(self.stacked_images)

# Model

In [9]:
class DenseNet121(nn.Module):
    def __init__(self):
        super(DenseNet121, self).__init__()
        self.model = torchvision.models.densenet121(pretrained = True)
        num_ftrs = self.model.classifier.in_features
        self.model.features[0] = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.model.classifier = nn.Sequential(
            nn.Linear(num_ftrs, nClasses),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.model(x)
        return x

In [10]:
data = DataPreprocessing()

stopHere =1
train_set, test_set = random_split(data, [math.ceil(len(data) * trainTestSplit), math.floor(round(len(data) * (1-trainTestSplit)))])

trainloader = torch.utils.data.DataLoader(train_set, batch_size=2, shuffle=trainShuffleFlag, num_workers=0)
testloader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False)

if device == "mps":
    model = DenseNet121().to(device)
    model = nn.DataParallel(model).to(device)
elif device == "cuda":
    model = DenseNet121().cuda()
    model = nn.DataParallel(model).cuda()
else :
    model = DenseNet121()
    model = nn.DataParallel(model)

c:\Users\Temp\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Temp\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
%%time

criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # RMSprop, Adam

for epoch in range(nEpochs): #loop over the dataset multiple times

    running_loss = 0.0
    correct = 0
    total = 0 
    for i, (images, labels) in enumerate(trainloader, 0): # get the inputs; data is a list of [images, labels]

        # zero the parameter gradients
        optimizer.zero_grad()
        
        if device == "mps":
            images = images.to(device)
        elif device == "cuda":
            images = images.cuda()
       
        #format input
        n_batches, n_crops, channels, height, width = images.size()
        image_batch = torch.autograd.Variable(images.view(-1, channels, height, width)) 
        
        if device == "mps":
            labels = tile(labels, 0, useImLevels).to(device) #duplicate for each crop the label 
        elif device == "cuda":
            labels = tile(labels, 0, useImLevels).cuda()
        else:
            labels = tile(labels, 0, useImLevels)
        
        # forward + backward + optimize
        outputs = model(image_batch)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()


        running_loss += loss.item()

        correct += compute_score_with_logits(outputs, labels).sum()
        total += labels.size(0)

    print('Epoch: %d, loss: %.3f, Accuracy: %.3f' %
          (epoch + 1, running_loss, 100 * correct / total))

print('Finished Training')


Epoch: 1, loss: 23.572, Accuracy: 55.664
Epoch: 2, loss: 20.987, Accuracy: 62.305
Epoch: 3, loss: 18.136, Accuracy: 72.461
Epoch: 4, loss: 16.501, Accuracy: 74.219
Epoch: 5, loss: 15.147, Accuracy: 73.633
Epoch: 6, loss: 14.414, Accuracy: 74.609
Epoch: 7, loss: 13.202, Accuracy: 76.367
Epoch: 8, loss: 12.582, Accuracy: 77.734
Epoch: 9, loss: 11.702, Accuracy: 80.078
Epoch: 10, loss: 10.691, Accuracy: 83.398
Epoch: 11, loss: 10.375, Accuracy: 84.570
Epoch: 12, loss: 9.329, Accuracy: 87.891
Epoch: 13, loss: 8.936, Accuracy: 88.867
Epoch: 14, loss: 8.011, Accuracy: 91.211
Epoch: 15, loss: 7.291, Accuracy: 92.969
Finished Training
CPU times: total: 9min 19s
Wall time: 5min 14s


In [17]:
model.eval()

DataParallel(
  (module): DenseNet121(
    (model): DenseNet(
      (features): Sequential(
        (conv0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu0): ReLU(inplace=True)
        (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (denseblock1): _DenseBlock(
          (denselayer1): _DenseLayer(
            (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace=True)
            (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace=True)
            (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          )
          (denselayer2): _DenseLayer

In [25]:
correct = 0
total = 0
with torch.no_grad():
    for i, (images, labels) in enumerate(testloader, 0):
        if device == "mps":
            images = images.to(device)
            labels = tile(labels, 0, useImLevels).to(device)
        elif device == "cuda":
            images = images.cuda()
            labels = tile(labels, 0, useImLevels).cuda()
        else:
            labels = tile(labels, 0, useImLevels)
        n_batches, n_crops, channels, height, width = images.size()
        image_batch = torch.autograd.Variable(images.view(-1, channels, height, width))
      
        outputs = model(image_batch)
        fitScore = compute_score_with_logits(outputs, labels).sum()
        correct += fitScore.item()
        total += labels.size(0)
        print(i+1, fitScore.item())
    
print('Accuracy on test set: %.3f' % (100 * correct / total))

1 8.0
2 8.0
3 3.0
4 8.0
5 8.0
6 8.0
7 8.0
8 8.0
9 8.0
10 8.0
11 4.0
12 8.0
13 8.0
14 7.0
15 8.0
16 8.0
Accuracy on test set: 92.188
